### bulk 火山图

In [ ]:
library(ggplot2)
library(dplyr)

# 读取 CSV
df <- read.csv("MvsK.csv")

# 检查列名
head(df)


In [ ]:
df <- df %>%
  mutate(
    Significance = case_when(
      Padjust < 0.05 & Log2FC > 2 ~ "Up",
      Padjust < 0.05 & Log2FC < -2 ~ "Down",
      TRUE ~ "NS"
    )
  )


In [ ]:
ggplot(df, aes(x = Log2FC, y = -log10(Padjust), color = Significance)) +
  geom_point(alpha = 0.6, size = 2) +
  scale_color_manual(values = c("Up"="red", "Down"="blue", "NS"="grey")) +
  theme_minimal() +
  xlab("Log2 Fold Change (LPS/CTRL)") +
  ylab("-log10(Padj)") +
  geom_vline(xintercept = c(-1,1), linetype="dashed", color="black") +
  geom_hline(yintercept = -log10(0.05), linetype="dashed", color="black") +
  theme(
    legend.title = element_blank(),
    axis.text = element_text(size=16),
    axis.title = element_text(size=16)
  )


In [ ]:
library(ggrepel)
options(repr.plot.width=10, repr.plot.height=8)
pdf("Volcano_MvsK.pdf", width=10, height=8)
top_genes <- df %>% filter(Padjust < 0.01 & abs(Log2FC) > 2)

ggplot(df, aes(x = Log2FC, y = -log10(Padjust), color = Significance)) +
  geom_point(alpha = 0.6, size = 2) +
  geom_text_repel(data = top_genes, aes(label = Gene_name), size=4) +
  scale_color_manual(values = c("Up"="red", "Down"="blue", "NS"="grey")) +
  theme_minimal() +
  xlab("Log2 Fold Change (M/K)") +
  ylab("-log10(Padj)") +
  geom_vline(xintercept = c(-2,2), linetype="dashed", color="black") +
  geom_hline(yintercept = -log10(0.05), linetype="dashed", color="black") +
  theme(
    axis.text = element_text(size = 14),      # XY轴刻度字体大小
    axis.title = element_text(size = 16),     # XY轴标题字体大小
    legend.text = element_text(size = 12),    # 图例文字大小
    legend.title = element_text(size = 14)    # 图例标题大小
  )
dev.off()

## 三个数据集的venn图!!!!

In [ ]:


library(dplyr)
library(readxl)
library(VennDiagram)

# 1. 读取 markers 文件
markers <- read_excel("06.markers_just_macro.xlsx")

# 选择 cluster = c10 且 avg_log2FC > 1 的基因
cluster2_genes <- markers %>%
  filter(cluster == "Macrophages_c10", avg_log2FC > 1) %>%
  pull(gene)

# 2. 读取差异基因 CSV 文件
df <- read.csv("MvsK.csv")

low_fc_genes <- df %>%
  filter(Log2FC > 2, Padjust < 0.05) %>%
  pull(Gene_name)

# 3. 读取 SCENIC 文件的 TargetGene 列
scenic <- read.csv("c10_scenic_gene_target.csv")

scenic_genes <- scenic %>%
  pull(TargetGene)

# 4. 绘制三集合 Venn 图
venn.plot <- venn.diagram(
  x = list(
    Cluster2 = cluster2_genes,
    LowFC    = low_fc_genes,
    SCENIC   = scenic_genes
  ),
  filename = NULL,
  fill = c("cornflowerblue", "darkorchid1", "seagreen3"),
  cat.col = c("cornflowerblue", "darkorchid1", "seagreen3"),
  cat.cex = 1.1,
  cat.pos = c(-20, 20, 180)
)

pdf(file="c10_up_3sets.pdf", width=6, height=6)
grid.draw(venn.plot)
dev.off()


In [ ]:
# 计算三个集合的交集基因
common_genes <- Reduce(intersect, list(cluster2_genes, low_fc_genes, scenic_genes))

# 找出交集基因在 MvsK.csv 中的 Log2FC 和 Padjust
common_df <- df %>%
  filter(Gene_name %in% common_genes) %>%
  select(Gene_name, Log2FC, Padjust) %>%
  arrange(desc(Log2FC))   # 按 Log2FC 从大到小排序

# 打印结果
print(common_df)

# 保存到文件
write.csv(common_df, "c10_common_gene_Log2FC_3sets.csv", row.names = FALSE)


## 二个数据集的venn图!!!!（bulk & sigmarker）

In [ ]:
# library(dplyr)
# library(readxl)
# library(VennDiagram)

# # 1. 读取 markers 文件
# markers <- read_excel("06.markers_just_macro.xlsx")

# # 选择 cluster = 2 且 |avg_log2FC| > 2 的基因
# cluster2_genes <- markers %>%
#   filter(cluster == "Macrophages_c10", avg_log2FC > 1) %>%
#   pull(gene)   # 提取 gene 列

# # 2. 读取差异基因 CSV 文件
# df <- read.csv("MvsK.csv")


# low_fc_genes <- df %>%
#   filter(Log2FC > 2, Padjust < 0.05) %>%
#   pull(Gene_name)   # 提取 Gene 列

# # 3. 绘制 Venn 图
# venn.plot <- venn.diagram(
#   x = list(
#     Cluster2 = cluster2_genes,
#     LowFC = low_fc_genes
#   ),
#   filename = NULL,
#   fill=c("cornflowerblue", "darkorchid1"),
# #   alpha = 0.5,
# #   cex = 2,
# cat.col = c("cornflowerblue", "darkorchid1"),
#   cat.cex = 1.1,
#   cat.pos = c(-1, 1),
# #   main = "Venn Diagram"
# )

# pdf(file="c10_up.pdf", width=5, height=5)
# grid.draw(venn.plot)
# dev.off()

In [ ]:
# # 找出交集基因在 MvsK.csv 中的 Log2FC 和 Padjust
# common_df <- df %>%
#   filter(Gene_name %in% common_genes) %>%
#   select(Gene_name, Log2FC, Padjust) %>%
#   arrange(desc(Log2FC))   # 按 Log2FC 升序排列（从小到大）

# # 打印结果
# print(common_df)

# # 保存到文件
# write.csv(common_df, "c10_common_gene_Log2FC_2.csv", row.names = FALSE)


### 交集基因在所有细胞亚型中得表达量气泡图

In [ ]:
library(Seurat)
library(dplyr)
library(patchwork)
# library(readr)
library(ggplot2)
#有云服务器的，可开启并运算，这里我用4个线程：
library(future)
library(qs)
# check the current active plan
plan()
# change the current plan to access parallelization
plan("multisession", workers =40)
plan()

#设置可用的内存
options(future.globals.maxSize = 4 * 1024^3)
plan("sequential")
future::plan()


In [ ]:
# seurat.data=qread(file = "/home/guoliming/Brown/ALI_CRL/ALI/Outdata/Step3.Cluster_annotion.qs")
# other_cells  <- subset(seurat.data, subset = celltype %in% c("T cells","B cells","Other Myeloid cells","Smooth muscle cells","Platelets"))

# Endothelial <- qread("/home/guoliming/Brown/ALI_CRL/ALI/Endothelial/Outdata/Sub_annotion.qs")
# Epithelial <- qread("/home/guoliming/Brown/ALI_CRL/ALI/Epithelial/Outdata/Sub_annotion.qs")
# Fibroblast <- qread("/home/guoliming/Brown/ALI_CRL/ALI/Fibroblast/Outdata/Sub_annotion.qs")

# NK_cells <- qread("/home/guoliming/Brown/ALI_CRL/ALI/NK cells/Outdata/Sub_annotion.qs")
# Neutrophils <- qread("/home/guoliming/Brown/ALI_CRL/ALI/Neutrophils/Outdata/Sub_annotion.qs")
# Macrophages <- qread("/home/guoliming/Brown/ALI_CRL/ALI/Macrophages/Outdata/Sub_annotion.qs")

# # 合并为一个新的 Seurat 对象
# seurat.data <- merge(other_cells, y = list(Endothelial, Epithelial,Fibroblast,NK_cells,Neutrophils,Macrophages), project = "ALI_Selected")

# seurat.data <- subset(seurat.data, subset = !grepl("\\(", celltype))

# qsave(seurat.data, file = "/home/guoliming/Brown/ALI_Gaoji/sc_bulk_analysis_Epithelial/7_RNAseq/all_cells.qs")

In [ ]:
seurat.data = qread(file = "/home/user/ALI/6_Bulk_RNAseq/all_cells.qs")

In [ ]:
common_genes = c(
"Cxcl9",
"Clec4e",
"Hmox1",
"Nos2",
"Sod2",
"Acod1",
"Tfec",
"Csf2rb",
"Msr1")


In [ ]:

# #绘制marker在各个cluster的气泡图
pdf(file="common_genes_Bubble_logfc1.pdf",width=20,height=12)
cluster10Marker=common_genes
DotPlot(object = seurat.data, features = cluster10Marker) + 
# RotatedAxis() +  # 旋转 X 轴文字，推荐 Seurat 自带函数
theme(
    axis.text.x = element_text(size = 16, angle = 45, hjust = 1),  # 横轴文字
    axis.text.y = element_text(size = 16),                          # 纵轴文字
    axis.title.x = element_blank(),  # 去掉X轴标题
    axis.title.y = element_blank(),  # 去掉Y轴标题
    legend.title = element_text(size = 16),
    legend.text  = element_text(size = 16),
    legend.key.size = unit(0.8, "cm")  # 图例符号大小
  )
dev.off()
